# A Stack Overflow Story

## Lab Assignment Two: Exploring Text Data 

### Justin Ledford, Luke Wood, Traian Pop 

In [13]:
import requests
import pprint
pp = pprint.PrettyPrinter(indent=2)
#BASEURL = "https://api.stackexchange.com/2.2/questions?site=stackoverflow&key=ckDNIw23urCJMtURIGBSIQ(("
#/2.2/questions?fromdate=1451606400&todate=1483228800&order=desc&sort=activity&site=stackoverflow
BASEURL="https://api.stackexchange.com/2.2/questions"
params = {
  "site" : "stackoverflow",
  "key" : "ckDNIw23urCJMtURIGBSIQ((",
  "fromdate" : "1451606400",
  "todate" : "1483228800",
  "order" : "desc",
  "sort" : "activity",
  "filter" : "withbody" #shows question and answer text
}

r = requests.get(BASEURL, params=params)

questions = r.json()

pp.pprint(questions)


{ 'has_more': True,
  'items': [ { 'accepted_answer_id': 38078456,
               'answer_count': 3,
               'body': '<p>I am stuck on creating azure batch pool with .net '
                       '4.6.1.</p>\n'
                       '\n'
                       '<p>I went through those very good resources:</p>\n'
                       '\n'
                       '<ul>\n'
                       '<li><p><a '
                       'href="https://social.msdn.microsoft.com/Forums/en-US/060e1d9e-d95c-454b-b693-cdec46d66919/compute-node-install-net-461?forum=azurebatch" '
                       'rel="nofollow noreferrer">Compute Node - Install .NET '
                       '4.6.1</a> </p></li>\n'
                       '<li><p><a '
                       'href="https://social.msdn.microsoft.com/Forums/en-US/4daf4077-ca94-435b-b4d9-a93f67993a06/how-to-get-the-windows-2016-preview-os-in-createpool?forum=azurebatch" '
                       'rel="nofollow noreferrer">How to get the Wind

In [19]:
from bs4 import BeautifulSoup
question_list = [x['body'] for x in questions['items']]
for x in question_list:
    print (BeautifulSoup(x, 'html.parser').code)
    print ("---------------------------\n")


I am stuck on creating azure batch pool with .net 4.6.1.
I went through those very good resources:

Compute Node - Install .NET 4.6.1 
How to get the Windows 2016 Preview OS

and I found out that there is a way to use .net 4.6.1 without installing it manually on a node. There is a programmatic way to set up Windows Ghost image with the latest .net version which is .net 4.6.1 but my node is defined in advance.

I need to have .net 4.6.1 because we use Data Factory and custom activities which are run on azure batch nodes. We upgraded .net version manually on that node but Microsoft doesn't guarantee that the state will preserve and we noticed a few times that node was reset to its original state.
My questions are:

do you know if there is any way on azure portal to choose OS family set to Windows Server 2016 with installed .net 4.6.1 version already?
does anybody know when there will be any os with .net 4.6.1 available?
or should I go with StartTask feature and try to install .net 4.6.1 